In [1]:
"""
Script d'annotation automatique pour des synopsis de films d'horreur.

Ce script lit les fichiers texte d'un corpus (originaux + versions augmentées),
et attribue à chaque synopsis des étiquettes (labels) basées sur la présence
de mots-clés issus d'un dictionnaire spécialisé "horreur".

Il produit un fichier CSV structuré contenant pour chaque texte :
- Le nom de fichier
- Le texte lui-même
- Le nombre de mots-clés horreur détectés
- Deux labels : 
    * label_binary (0 = non-horreur, 1 = horreur)
    * label_severity (0 = neutre, 1 = modéré, 2 = intense) (s'il y a beaucoup d'élements d'horreur ou non )

Utilisation :
-------------
Ce script est conçu pour compléter une tâche d'enrichissement de données (data augmentation).

Fonctionnalités principales :
-----------------------------
1. Analyse lexicale à partir d’un dictionnaire de mots-clés horreur.
2. Comptage des mots-clés présents dans chaque fichier.
3. Attribution de labels en fonction du score.
4. Agrégation des résultats dans un DataFrame et export en CSV.

Pré-requis :
------------
- Python 3.x
- pandas (pip install pandas)

Entrées attendues :
-------------------
- Dossiers contenant les synopsis originaux et augmentés :
    * original_dir = "../film_synopses"
    * augmented_dir = "synopsis_augmente"

Sortie :
--------
- Un fichier CSV nommé `dataset_horreur.csv` contenant toutes les annotations.
"""

import pandas as pd
import os
keywords = set([
    "esprit", "fantome", "demon", "possede", "malediction", "sorcellerie", "rituel", "exorcisme", "enfer", "pacte", "apocalypse",
    "tueur", "meurtre", "massacre", "sacrifice", "torture", "sang", "cadavre", "hurlement", "agonie", "dechiquete", "mutilation",
    "peur", "terreur", "cauchemar", "angoisse", "panique", "crie", "epouvante", "frisson", "folie", "paranoia", "hallucination",
    "maison", "manoir", "hopital", "asile", "foret", "cimetiere", "grenier", "cave", "miroir", "poupee", "croix", "talisman",
    "monstre", "zombie", "vampire", "loup-garou", "momie", "clown", "chasseur", "entite", "ombre", "spectre", "revenant",
    "enfant", "bebe", "innocent", "orphelin", "famille", "adoption",
    "ame errante", "presence invisible", "apparition", "entite malefique", "creature infernale", "diable", "habite", "controle", "sous emprise",
    "sort", "condamnation", "fleau mystique", "magie noire", "sabbat", "envoutement", "ceremonie occulte", "invocation", "rite ancien",
    "purification", "chasse aux demons", "bannissement", "abime", "royaume infernal", "gehenne", "accord maudit", "serment demoniaque", "marche noir",
    "assassin", "psychopathe", "meurtrier", "homicide", "crime sanglant", "assassinat", "boucherie", "tuerie", "hecatombe",
    "offrande sanglante", "immolation", "rite de sang", "supplice", "souffrance", "agonie lente",
    "trainee sanglante", "fluide vital", "giclee rouge", "corps sans vie", "depouille", "restes humains",
    "cri strident", "appel a l'aide", "sanglot dechirant", "souffrance finale", "derniers instants", "lente douleur",
    "demembre", "eventre", "lacere", "amputation", "defiguration", "blessure extreme",
    "fin du monde", "jugement dernier", "cataclysme",
    "effroi", "frayeur", "angoisse", "rêve macabre", "hallucination nocturne", "vision d'horreur",
    "stress profond", "peur viscérale", "malaise constant", "detresse", "affolement", "crise de peur",
    "hurle", "braille", "gemit", "horreur", "effroi pur", "terreur froide",
    "chair de poule", "tremblement", "sensation glaciale", "delire", "alienation", "perte de raison",
    "mefiance extreme", "delire de persecution", "obsession", "vision irreelle", "illusion", "derive mentale",
    "demeure sinistre", "batisse hantee", "chateau lugubre", "villa hantee", "vieille batisse",
    "centre abandonne", "clinique vide", "hopital psychiatrique", "institution fermee", "prison mentale",
    "bois obscur", "sous-bois hantee", "jungle maudite", "necropole", "lieu de repos eternel", "terre des morts",
    "piece oubliee", "recoin poussiereux", "etage noir", "sous-sol", "cellier lugubre", "souterrain",
    "reflet maudit", "verre hante", "surface trompeuse", "jouet malefique", "figurine animee", "objet possede",
    "symbole sacre", "croix inverse", "relique", "amulette", "objet mystique", "artefact sacre",
    "creature", "abomination", "bete difforme", "mort-vivant", "cadavre anime", "creature decomposee",
    "buveur de sang", "sanguinaire", "immortel nocturne", "bete lunaire", "homme-loup", "creature lycanthrope",
    "corps embaume", "ancien roi maudit", "etre bande", "bouffon malefique", "farceur tordu", "joker demoniaque",
    "traqueur", "pisteur", "tueur silencieux", "forme invisible", "creature inconnue", "presence paranormale",
    "silhouette noire", "absence de lumiere", "forme mouvante", "vision etheree", "ame en peine", "fantome du passe", "mort revenu",
    "ame innocente", "orphelin hante", "petit etre fragile", "nouveau-ne", "poupon", "petit etre",
    "pur", "candide", "naif", "sans famille", "enfant abandonne", "isole",
    "proches", "parents", "lignee", "accueil", "prise en charge", "nouvelle famille"

])


# Fonction pour compter les mots-clés horrifiques dans un texte
def count_horror_keywords(text):
    # Convertit le texte en minuscules et le découpe en mots
    words = text.lower().split()
    # Compte le nombre de mots qui apparaissent dans la liste 'keywords'
    return sum(word in keywords for word in words)

# Fonction pour attribuer les labels binaires et de sévérité selon le nombre de mots-clés
def compute_labels(horror_count):
    if horror_count == 0:
        return 0, 0  # Aucun mot-clé => pas d'élements d’horreur
    elif horror_count <= 25:
        return 1, 1  # Faible présence => nombre d'élements d'horreur légère
    else:
        return 1, 2  # Forte présence => nombre d'élements d'horreur sévère

# Fonction principale pour construire le dataset final
def prepare_dataset(original_dir, augmented_dir, output_csv):
    data = []  # Liste pour stocker tous les exemples

    # Parcourt les deux répertoires (originaux et augmentés)
    for dir_path in [original_dir, augmented_dir]:
        for filename in os.listdir(dir_path):
            if filename.endswith(".txt"):  # Ne traite que les fichiers texte
                path = os.path.join(dir_path, filename)
                # Lit le contenu du fichier
                with open(path, "r", encoding="utf-8") as f:
                    text = f.read().strip()
                # Compte des mots-clés horrifiques
                horror_count = count_horror_keywords(text)
                # Attribution des deux types de labels
                label_binary, label_severity = compute_labels(horror_count)
        
                data.append({
                    "filename": filename,
                    "text": text,
                    "label_binary": label_binary,
                    "label_severity": label_severity,
                    "horror_keywords_count": horror_count    
                })

    # Conversion en DataFrame puis sauvegarde dans un fichier CSV
    df = pd.DataFrame(data)
    df.to_csv(output_csv, index=False, encoding='utf-8')
    print(f"Dataset sauvegardé dans {output_csv} — total {len(df)} exemples.")

# Point d’entrée du script si exécuté directement
if __name__ == "__main__":
    prepare_dataset("../film_synopses", "synopsis_augmente", "dataset_horreur.csv")

             


Dataset sauvegardé dans dataset_horreur.csv — total 96 exemples.
